## Santander Product Recommendation - Part 2
#### Part 2: Feature Database - Train/Val set Generation 
This is the work demo for Satandander Product Recommendation Project, which is a also Kaggle Contest. We ranked as 12nd in Public LB and 16th in Private LB. In this project the target was to recommend new products to customers based on their historical behavioral patterns, product purchase records as well as demographic information. The demo will give a step-by-step workflow of my work. Basically this notebook includes:
- Part 1 - Data cleaning
- Part 2 - Feature Bank Generation
- Part 3 - EDA and feature exploration
- Part 4 - Model Training and Validation

**Note:** *We only use training data provided for the demonstration and validation as the true label was not provided in test data*


In [115]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [116]:
## disable warnings
config_db = "../input/santander_full.sqlite"

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime
import gc
import seaborn as sns
import matplotlib.pyplot as plt
%pylab inline
pylab.rcParams['figure.figsize'] = (10, 6)

import sqlite3 as sq

## connect to database

sq_conn = sq.connect(config_db)

Populating the interactive namespace from numpy and matplotlib


In [118]:
## create modules for SQL commands
def create_simple_join(table_new, table_left, table_right, cols_left, cols_right, sufx_left, sufx_right, \
                       join_type = "LEFT JOIN", key_left = ["date_record", "cust_id"], key_right = ["date_record", "cust_id"], \
                       order_by = ["date_record", "cust_id"], null_as_zero = False):
    """
    create simple join command in SQL
    """    
        
    select_left = ','.join(["A." + col + " AS " + col for col in ["date_record", "cust_id"]]) + ',' + \
    ','.join(["A." + col + " AS " + col + sufx_left for col in cols_left]) if cols_left else \
    ','.join(["A." + col + " AS " + col for col in ["date_record", "cust_id"]])
    
    select_right = ','.join(["B." + x + " AS " + x + sufx_right for x in cols_right]) if not null_as_zero else \
        ','.join(["IFNULL(B." + x + ",0) AS " + x + sufx_right for x in cols_right])    
        
    cols_selected = select_left + ',' + select_right
        
    cmd = "CREATE TABLE "+ table_new +" AS SELECT"
    cmd += " " + cols_selected
    cmd += """ FROM """ +\
    table_left + " AS A " +\
    join_type + ' ' +\
    table_right + " AS B " +\
    """ ON """ +\
    " AND ".join(" A." + x + ' = ' + " B." + y for x, y in zip(key_left, key_right)) +\
    " ORDER BY " + ','.join(order_by) + ';'
            
    return cmd

def sql_create_table(table, cmd):
    """
    write sql query: create new table cmd
    """
    
    sq_cursor = sq_conn.cursor()
    sq_cursor.execute("DROP TABLE IF EXISTS " + table + ';')
    sq_cursor.execute(cmd)
    sq_conn.commit()


### 1. Create table from joining current month and previous month

In [119]:
## Define product feature columns
cols_product = ["p1_saving_acct", "p2_guarantees", "p3_cur_acct", "p4_deriv_acct", 
"p5_payroll_acct", "p6_junior_acct", "p7_mas_paticular_acct", "p8_particular_acct", "p9_particular_plus_acct", "p10_short_depo", 
"p11_medium_depo", "p12_long_depo", "p13_eacct", "p14_funds", "p15_mortgage", "p16_pensions_plan", "p17_loans", 
"p18_taxes", "p19_credit_card", "p20_securities", "p21_home_acct", "p22_payroll_appt", "p23_pensions_appt", "p24_direct_debit"]

In [120]:
## command to join current month's product w. previous month
cmd = create_simple_join(table_new = "train_join_cur_prev", table_left = "train_sample_clean", \
    table_right = "train_sample_clean", cols_left = cols_product,\
    cols_right = cols_product, sufx_left = "_cur", sufx_right = "_prev", \
    join_type = "LEFT JOIN", key_left = ["date_record - 1", "cust_id"], \
    key_right = ["date_record", "cust_id"], order_by = ["A.date_record", "A.cust_id"], null_as_zero = True)

In [122]:
sql_create_table("train_join_cur_prev", cmd)

In [123]:
train_join_cur_prev  = pd.read_sql_query("SELECT * FROM train_join_cur_prev;", sq_conn)

In [124]:
print train_join_cur_prev.shape
train_join_cur_prev.head()

(13647309, 50)


,date_record,cust_id,p1_saving_acct_cur,p2_guarantees_cur,p3_cur_acct_cur,p4_deriv_acct_cur,p5_payroll_acct_cur,p6_junior_acct_cur,p7_mas_paticular_acct_cur,p8_particular_acct_cur,...,p15_mortgage_prev,p16_pensions_plan_prev,p17_loans_prev,p18_taxes_prev,p19_credit_card_prev,p20_securities_prev,p21_home_acct_prev,p22_payroll_appt_prev,p23_pensions_appt_prev,p24_direct_debit_prev
0,0,15889,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,15890,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,15892,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,15893,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,15894,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Create Label dataset: original and stacked

**Data filtering: keep users that only purchased new product for each month**

The stacked dataset is generated from original by duplicating rows with multiple product purchase, so that each row correspond to only one product purchase. 

#### 2.1 Create label table: the new purchased product each month - user

```
train_active_label (unstacked) = ["num_new_products" + "new_products"] + [x + "_new" for x in cols_product]
```

In [125]:
def create_label():
    """
    generate SQL command for creating label related features
    """
    
    cols_selected = "date_record, cust_id, " +\
    '+'.join(["MAX(0, " + x + "_cur" + '-' + x + "_prev)" for x in cols_product]) +\
    " AS " + "num_new_products" + ',' + ','.join(["MAX(0, " + x + "_cur" + '-' + x + "_prev) AS " + x + "_new" for x in cols_product]) 
            
    cmd = "CREATE TABLE train_active_label AS SELECT"
    cmd += " " + cols_selected
    cmd += """ 
    FROM 
    train_join_cur_prev
    WHERE
    num_new_products > 0 AND date_record > 0
    ORDER BY date_record, cust_id;
    """
            
    return cmd        

In [127]:
sql_create_table("train_active_label", cmd)

In [128]:
train_active_label = pd.read_sql_query("SELECT * FROM train_active_label;", sq_conn)
print train_active_label.shape
train_active_label.head()

(547571, 27)


,date_record,cust_id,num_new_products,p1_saving_acct_new,p2_guarantees_new,p3_cur_acct_new,p4_deriv_acct_new,p5_payroll_acct_new,p6_junior_acct_new,p7_mas_paticular_acct_new,...,p15_mortgage_new,p16_pensions_plan_new,p17_loans_new,p18_taxes_new,p19_credit_card_new,p20_securities_new,p21_home_acct_new,p22_payroll_appt_new,p23_pensions_appt_new,p24_direct_debit_new
0,1,15924,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,15925,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,15928,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,15965,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,16026,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [129]:
## Add column "new_products" = list of product indices (from 0 to 23)
#### 1. create df_prod_idx: the row/col indices for non-zero products

prod_idx = train_active_label.loc[:, [x + "_new" for x in cols_product]].values.nonzero()
df_prod_idx = pd.DataFrame(dict(zip(["idx_row", "idx_col"], list(prod_idx))))
df_prod_idx_list = df_prod_idx.groupby("idx_row").agg(lambda x: list(x)).reset_index()
df_prod_idx_list.rename(columns = {"idx_row": "index", "idx_col": "new_products"}, inplace = True)

print df_prod_idx.shape, df_prod_idx_list.shape
print df_prod_idx.head()
print df_prod_idx_list.head()

(669508, 2) (547571, 2)
   idx_col  idx_row
0       18        0
1       23        1
2       18        2
3        7        3
4        8        3
   index new_products
0      0         [18]
1      1         [23]
2      2         [18]
3      3       [7, 8]
4      4         [18]


In [130]:
#### 2. Join df_prod_idx_list onto train_active_label
train_active_label.reset_index(inplace = True) # create explicit index
train_active_label = train_active_label.merge(df_prod_idx_list, how = "left", on = "index")

In [131]:
train_active_label.loc[:, "new_products"] = train_active_label.loc[:, "new_products"].apply(str)
print train_active_label.shape
train_active_label.head()

(547571, 29)


,index,date_record,cust_id,num_new_products,p1_saving_acct_new,p2_guarantees_new,p3_cur_acct_new,p4_deriv_acct_new,p5_payroll_acct_new,p6_junior_acct_new,...,p16_pensions_plan_new,p17_loans_new,p18_taxes_new,p19_credit_card_new,p20_securities_new,p21_home_acct_new,p22_payroll_appt_new,p23_pensions_appt_new,p24_direct_debit_new,new_products
0,0,1,15924,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]
1,1,1,15925,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[23]
2,2,1,15928,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]
3,3,1,15965,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[7, 8]"
4,4,1,16026,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]


In [132]:
## write train_active_label into sql
train_active_label.to_sql(name='train_active_label', con=sq_conn, if_exists='replace', index=False, index_label=None)

#### 2.2 Create stacked training label dataset: repeat for each purchased product
The purpose is to provide easier data processing for EDA - Feature Engineering, as well as model training later

```
train_active_label_stacked = ["num_new_products" + "new_products" + "label"] + [x + "_new" for x in cols_product]
```

**Note that label = index of the unique new product in stacked data (from 0 to 23)**

In [133]:
## 1. Create desired product value
prod_arr = np.zeros((df_prod_idx.shape[0], len(cols_product))).astype(int)
prod_arr[range(df_prod_idx.shape[0]), df_prod_idx.loc[:, "idx_col"].values] = 1

In [136]:
## 2. Join unstacked train_active_label onto df_prod_idx to form the stacked train_active_label_stacked
train_active_label_stacked = df_prod_idx.merge(train_active_label, how = "left", left_on="idx_row", right_on="index")
train_active_label_stacked.loc[:, [x + '_new' for x in cols_product]] = prod_arr

In [137]:
print train_active_label_stacked.shape
train_active_label_stacked.head()

(669508, 31)


,idx_col,idx_row,index,date_record,cust_id,num_new_products,p1_saving_acct_new,p2_guarantees_new,p3_cur_acct_new,p4_deriv_acct_new,...,p16_pensions_plan_new,p17_loans_new,p18_taxes_new,p19_credit_card_new,p20_securities_new,p21_home_acct_new,p22_payroll_appt_new,p23_pensions_appt_new,p24_direct_debit_new,new_products
0,18,0,0,1,15924,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]
1,23,1,1,1,15925,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[23]
2,18,2,2,1,15928,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]
3,7,3,3,1,15965,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[7, 8]"
4,8,3,3,1,15965,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[7, 8]"


In [138]:
## rename the "idx_col" into "label"
train_active_label_stacked.rename(columns = {"idx_col": "label"}, inplace = True)
train_active_label_stacked.drop(["idx_row", "index"], axis = 1, inplace = True)


In [139]:
print train_active_label_stacked.shape
train_active_label_stacked.head()

(669508, 29)


,label,date_record,cust_id,num_new_products,p1_saving_acct_new,p2_guarantees_new,p3_cur_acct_new,p4_deriv_acct_new,p5_payroll_acct_new,p6_junior_acct_new,...,p16_pensions_plan_new,p17_loans_new,p18_taxes_new,p19_credit_card_new,p20_securities_new,p21_home_acct_new,p22_payroll_appt_new,p23_pensions_appt_new,p24_direct_debit_new,new_products
0,18,1,15924,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]
1,23,1,15925,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,[23]
2,18,1,15928,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,[18]
3,7,1,15965,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[7, 8]"
4,8,1,15965,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[7, 8]"


In [140]:
## 3. Import table train_active_label_stacked onto SQL 
train_active_label_stacked.to_sql(name='train_active_label_stacked', \
                                  con=sq_conn, if_exists='replace', index=False, index_label=None)

### 3. Join Features onto Label dataset (**Unstacked ONLY**)


#### 3.1 Customer Profile Features current month **Unstacked ONLY**

```
train_active_label_profile = train_active_label + ['idx_employee', 'country_code', 'sex', 'age', 'date_first_holder', 'idx_new_cust', 'months_seniority', 'idx_primary', 'type_cust', 'type_cust_relation', 'idx_same_country', 'idx_foreigner', 'channel', 'idx_decease', 'prov_name', 'idx_active', 'income', 'segmentation']
```


In [141]:
## define profile features
cols_profile = ['idx_employee', 'country_code', 'sex', 'age', 'date_first_holder', \
        'idx_new_cust', 'months_seniority', 'idx_primary', 'type_cust', 'type_cust_relation', \
        'idx_same_country', 'idx_foreigner', 'channel', 'idx_decease', 'prov_name', 'idx_active', \
        'income', 'segmentation']

In [142]:
## generate train_active_label_profile (unstacked)
cmd = create_simple_join(table_new = "train_active_label_profile", table_left = "train_active_label", \
    table_right = "train_sample_clean", cols_left = [],\
    cols_right = cols_profile, sufx_left = '', sufx_right = '', \
    join_type = "LEFT JOIN", key_left = ["date_record", "cust_id"], \
    key_right = ["date_record", "cust_id"], order_by = ["A.date_record", "A.cust_id"], null_as_zero = False)

print cmd

CREATE TABLE train_active_label_profile AS SELECT A.date_record AS date_record,A.cust_id AS cust_id,B.idx_employee AS idx_employee,B.country_code AS country_code,B.sex AS sex,B.age AS age,B.date_first_holder AS date_first_holder,B.idx_new_cust AS idx_new_cust,B.months_seniority AS months_seniority,B.idx_primary AS idx_primary,B.type_cust AS type_cust,B.type_cust_relation AS type_cust_relation,B.idx_same_country AS idx_same_country,B.idx_foreigner AS idx_foreigner,B.channel AS channel,B.idx_decease AS idx_decease,B.prov_name AS prov_name,B.idx_active AS idx_active,B.income AS income,B.segmentation AS segmentation FROM train_active_label AS A LEFT JOIN train_sample_clean AS B  ON  A.date_record =  B.date_record AND  A.cust_id =  B.cust_id ORDER BY A.date_record,A.cust_id;


In [143]:
## generate train_active_label_profile (unstacked)
sql_create_table("train_active_label_profile", cmd)

In [144]:
train_active_label_profile  = pd.read_sql_query("SELECT * FROM train_active_label_profile;", sq_conn)
print train_active_label_profile.shape
train_active_label_profile.head()
#train_active_label_profile.tail()
# print train_active_label_profile.isnull().sum()

(547571, 20)


,date_record,cust_id,idx_employee,country_code,sex,age,date_first_holder,idx_new_cust,months_seniority,idx_primary,type_cust,type_cust_relation,idx_same_country,idx_foreigner,channel,idx_decease,prov_name,idx_active,income,segmentation
0,1,15924,B,ES,V,52.0,1995-01-16,0,246.0,1,1,A,S,N,KAT,N,"PALMAS, LAS",1,130903.0,02 - PARTICULARES
1,1,15925,F,ES,V,62.0,1995-01-16,0,246.0,1,1,A,S,N,KAT,N,MADRID,1,42831.0,02 - PARTICULARES
2,1,15928,B,ES,H,49.0,1995-01-16,0,246.0,1,1,A,S,N,KAT,N,MADRID,1,263023.0,01 - TOP
3,1,15965,F,ES,H,51.0,1995-01-16,0,246.0,1,1,A,S,N,KAT,N,MADRID,1,44353.0,01 - TOP
4,1,16026,F,ES,H,53.0,1995-02-06,0,245.0,1,1,A,S,N,KAT,N,MADRID,1,151911.0,02 - PARTICULARES


#### 3.2 Lag product purchase features, lag 1 to 11 (current_month - 1, to current_month - 11) (Unstacked ONLY)
```
cols_product = ["p1_saving_acct", "p2_guarantees", "p3_cur_acct", "p4_deriv_acct", 
"p5_payroll_acct", "p6_junior_acct", "p7_mas_paticular_acct", "p8_particular_acct", "p9_particular_plus_acct", "p10_short_depo", 
"p11_medium_depo", "p12_long_depo", "p13_eacct", "p14_funds", "p15_mortgage", "p16_pensions_plan", "p17_loans", 
"p18_taxes", "p19_credit_card", "p20_securities", "p21_home_acct", "p22_payroll_appt", "p23_pensions_appt", "p24_direct_debit"]
```

**NOTE: ** the lag-0 product is the current month product purchase, should be used to create label, and not be used in feature

In [145]:
def create_product_lag(lag):
        
    """
    generate SQL command for creating lag product purchase features
    """
    
    cols_selected = "A.date_record AS date_record, A.cust_id AS cust_id, " +\
    ','.join(["IFNULL(B." + x + ", 0) AS " + x + "_lag_" + str(lag) for x in cols_product])   
    cond_lag_join = "A.date_record - " + str(lag) + " = B.date_record"
    
    cmd = "CREATE TABLE train_active_label_product_lag_" + str(lag) + " AS SELECT"
    cmd += " " + cols_selected
    cmd += """ 
    FROM 
    train_active_label AS A 
    LEFT JOIN 
    train_sample_clean AS B
    ON 
    A.cust_id = B.cust_id
    AND
    """ + \
    cond_lag_join + \
    """
    ORDER BY A.date_record, A.cust_id;
    """
                    
    return cmd

In [146]:
lag = 2
cmd = create_product_lag(lag)
print cmd

CREATE TABLE train_active_label_product_lag_2 AS SELECT A.date_record AS date_record, A.cust_id AS cust_id, IFNULL(B.p1_saving_acct, 0) AS p1_saving_acct_lag_2,IFNULL(B.p2_guarantees, 0) AS p2_guarantees_lag_2,IFNULL(B.p3_cur_acct, 0) AS p3_cur_acct_lag_2,IFNULL(B.p4_deriv_acct, 0) AS p4_deriv_acct_lag_2,IFNULL(B.p5_payroll_acct, 0) AS p5_payroll_acct_lag_2,IFNULL(B.p6_junior_acct, 0) AS p6_junior_acct_lag_2,IFNULL(B.p7_mas_paticular_acct, 0) AS p7_mas_paticular_acct_lag_2,IFNULL(B.p8_particular_acct, 0) AS p8_particular_acct_lag_2,IFNULL(B.p9_particular_plus_acct, 0) AS p9_particular_plus_acct_lag_2,IFNULL(B.p10_short_depo, 0) AS p10_short_depo_lag_2,IFNULL(B.p11_medium_depo, 0) AS p11_medium_depo_lag_2,IFNULL(B.p12_long_depo, 0) AS p12_long_depo_lag_2,IFNULL(B.p13_eacct, 0) AS p13_eacct_lag_2,IFNULL(B.p14_funds, 0) AS p14_funds_lag_2,IFNULL(B.p15_mortgage, 0) AS p15_mortgage_lag_2,IFNULL(B.p16_pensions_plan, 0) AS p16_pensions_plan_lag_2,IFNULL(B.p17_loans, 0) AS p17_loans_lag_2,IFNU

In [147]:
## Create product lag features 1 to 11
for lag in xrange(1, 12):
    cmd = create_product_lag(lag)
    table = "train_active_label_product_lag_" + str(lag)
    print "create table " + table        
    sql_create_table(table, cmd)

create table train_active_label_product_lag_1
create table train_active_label_product_lag_2
create table train_active_label_product_lag_3
create table train_active_label_product_lag_4
create table train_active_label_product_lag_5
create table train_active_label_product_lag_6
create table train_active_label_product_lag_7
create table train_active_label_product_lag_8
create table train_active_label_product_lag_9
create table train_active_label_product_lag_10
create table train_active_label_product_lag_11


### 4. Split train(stacked) and validation(unstacked)

**use the first 16 months (2015-01 to 2016-04) as train and the 17th month (2016-05) as validation, mimic the train/test split for the competition**



In [148]:
## unstacked train and val split
data_train_label_stacked = pd.read_sql_query("SELECT * FROM train_active_label_stacked where date_record < 16;", sq_conn)
data_val_label = pd.read_sql_query("SELECT * FROM train_active_label where date_record = 16;", sq_conn)
print data_train_label_stacked.shape, data_val_label.shape
print data_train_label_stacked.head()
print data_val_label.head()

(631611, 29) (29717, 29)
   label  date_record  cust_id  num_new_products  p1_saving_acct_new  \
0     18            1    15924                 1                   0   
1     23            1    15925                 1                   0   
2     18            1    15928                 1                   0   
3      7            1    15965                 2                   0   
4      8            1    15965                 2                   0   

   p2_guarantees_new  p3_cur_acct_new  p4_deriv_acct_new  p5_payroll_acct_new  \
0                  0                0                  0                    0   
1                  0                0                  0                    0   
2                  0                0                  0                    0   
3                  0                0                  0                    0   
4                  0                0                  0                    0   

   p6_junior_acct_new      ...       p16_pensions_plan_

### 5. Join profile and lag 1-11 product features onto train (stacked) and val (unstacked)

In [149]:
## Use pandas merge to do recursive join
#### Join profile
train_active_label_profile = pd.read_sql_query("SELECT * FROM train_active_label_profile;", sq_conn)
data_train_label_stacked = data_train_label_stacked.merge(train_active_label_profile, \
                                                          how = "left", on = ["date_record", "cust_id"])
data_val_label = data_val_label.merge(train_active_label_profile, how = "left", on = ["date_record", "cust_id"])

In [153]:
print data_train_label_stacked.shape, data_val_label.shape
data_train_label_stacked.head()

(631611, 311) (29717, 311)


,label,date_record,cust_id,num_new_products,p1_saving_acct_new,p2_guarantees_new,p3_cur_acct_new,p4_deriv_acct_new,p5_payroll_acct_new,p6_junior_acct_new,...,p15_mortgage_lag_11,p16_pensions_plan_lag_11,p17_loans_lag_11,p18_taxes_lag_11,p19_credit_card_lag_11,p20_securities_lag_11,p21_home_acct_lag_11,p22_payroll_appt_lag_11,p23_pensions_appt_lag_11,p24_direct_debit_lag_11
0,18,1,15924,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,23,1,15925,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18,1,15928,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,1,15965,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,1,15965,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
#### Join lag 1 - 11 product
for lag in xrange(1, 12):
    print lag
    data_tmp_lag = pd.read_sql_query("SELECT * FROM train_active_label_product_lag_" + str(lag) + ';', sq_conn)
    data_train_label_stacked = data_train_label_stacked.merge(data_tmp_lag, how = "left", on = ["date_record", "cust_id"])
    data_val_label = data_val_label.merge(data_tmp_lag, how = "left", on = ["date_record", "cust_id"])

1
2
3
4
5
6
7
8
9
10
11
